In [1]:
import numpy as np
from numpy.linalg import norm
import scipy
from scipy.optimize.linesearch import line_search_armijo

/var/folders/k5/vn3k1nw15mv2z0p8ww8lv16r0000gn/T/ipykernel_5382/584618614.py:4: DeprecationWarning: Please use `line_search_armijo` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprecated.
  from scipy.optimize.linesearch import line_search_armijo


## Steepest Descent Algorithm

In [2]:
def gradient_descent(x0,
                     cost_function,
                     gradient_function,
                     threshold = 1e-8, 
                     step_size = 1e-4, 
                     log = True):
    # initialize the counter    
    i = 0
    # initialize x
    x = x0
    # compute the first gradient
    gradient = gradient_function(x)
    # continue to iterate while the norm of the 
    # gradient is greater than the threshold
    while norm(gradient) >= threshold: 
        # update the gradient
        gradient = gradient_function(x)
        # move to a new x by moving from the original x in the negative
        # direction of the gradient according to a given step size
        x = x - step_size*gradient
        # compute the cost at the new x and update the minimum cost (shouldn't we confirm new cost is smaller?)
        minimum = cost_function(x)
        # update the counter
        i += 1
        # print the value of x and the cost at x every 1000 iterations
        if log and i % 1e4 == 0: 
            print(f'x = {x}, V(x) = {minimum:.5f}')
    # return x and the corresponding minimum cost
    return x, minimum


## Armijo Step Size Selection

In [3]:
def armijo(x, cost_function, gradient, search_dir, gamma=1.5, r = 0.8, log=True):
    def v_bar(cost_x,grad_x_s,w):
        return cost_x + 0.5*w*grad_x_s
    w = 0.1
    cost_x = cost_function(x)
    grad_x_s = gradient @ search_dir
    # initialize p
    p = 0
    # propogate forward
    while cost_function(x + (gamma**p)*search_dir) < v_bar(cost_x, grad_x_s, (gamma**p)): 
        w = gamma**p
        # increment p
        p += 1
    # initialize q
    q = 0
    # propogate backwards
    while cost_function(x + (r**q * gamma**p)*search_dir) > v_bar(cost_x, grad_x_s, r**q * gamma**p): 
        # consider step size w
        w = r**q * gamma**p
        # increment q
        q += 1
    # return step size
    if log:
        print(f'p={p}, q={q}, w={w}')
    return w


## Conjugate Gradient Algorithm

In [4]:
def conjugate_gradient(x0,
                     cost_function,
                     gradient_function,
                     threshold = 1e-8, 
                     step_size = 1e-4,
                     max_iter = 9999,
                     log = True):
    i = 0
    prev_gradient = gradient_function(x0)
    search_direction = prev_gradient * -1
    while norm(prev_gradient) >= threshold: 
        #add armijo step size
        #step_size = armijo(x0, cost_function, prev_gradient, search_dir=-np.ones_like(x0), gamma=1.5, r=0.8, log=False)
        #step_size = line_search_armijo(cost_function, x0, np.ones_like(x0)*-1, prev_gradient, cost_function(x0))[0]
        x1 = x0 + step_size * search_direction
        next_gradient = gradient_function(x1)
        beta = np.dot((next_gradient - prev_gradient) , next_gradient)
        beta /= np.dot(prev_gradient,prev_gradient)
        search_direction = -1*next_gradient + beta * search_direction
        prev_gradient = next_gradient
        x0 = x1
        minimum = cost_function(x0)
        i += 1
        if log and i%1e4 == 0: 
            print(f'x = {x0}, V(x) = {minimum:.5f}')
        # if i > max_iter:
        #     break
    return x0, minimum

## Secant Algorithm

In [9]:
def secant(x0,
            cost_function,
            gradient_function,
            threshold = 1e-6, 
            step_size = 1e-4,
            fixed_step = True):
    H = np.eye(len(x0))
    j=0
    while True:
        gradient_x0 = gradient_function(x0)
        s = -np.matmul(H,gradient_x0.reshape(-1,1))
        w = step_size if fixed_step else armijo(x0, cost_function, gradient_x0, search_dir=s.flatten(), gamma=1.5, r=0.8, log=False)
        x1 = x0 + w*s.flatten()
        gradient_x1 = gradient_function(x1)
        if norm(gradient_x1) < threshold:
            break
        dx = (x1-x0).reshape(-1,1)
        dg = (gradient_x1-gradient_x0).reshape(-1,1)
        H = H + np.matmul(dx,dx.reshape(1,-1))/np.dot(dx.reshape(1,-1),dg) - np.matmul(np.matmul(H,dg),(np.matmul(H,dg)).reshape(1,-1))/np.dot(dg.reshape(1,-1),np.matmul(H,dg))
        j += 1
        x0 = x1
        minimum  = cost_function(x0)
    return x0, minimum


## Finite Difference Approximation

Recall for a given partial derivative with respect to an element of $x$ (i.e $x_1$), the finitie difference approximation is as follows:

$\frac{\partial V}{\partial x_1}\left(\begin{bmatrix}x_1\\x_2\end{bmatrix}\right) \cong \frac{V\left(\begin{bmatrix}x_1 + h\\x_2\end{bmatrix}\right) - V\left(\begin{bmatrix}x_1\\x_2\end{bmatrix}\right)}{h}$

In [6]:
def gradient_approx(x0, cost_function, h):
    gradient = np.zeros_like(x0)
    perturbation = np.eye(len(x0))*h
    for i in range(len(x0)):
        gradient[i] = (cost_function(x0+perturbation[i])-cost_function(x0))/h
    return gradient

In [ ]:
print(gradient_approx(x0, V_a, h=0.01))
print(gradV_a(x0))

[ 28.74328766  -4.89440894 -16.65011163 -85.78824817 176.84435094
 139.88750454]
[ 28.65328766  -5.00440894 -16.78011163 -85.95824817 206.96526976
  96.37429349]


## Penalty and Barrier Functions

### Cost Function A

In [10]:
def V_a(x):
    a = np.array([5])
    b = np.array([1, 4, 5, 4, 2, 1])
    C = [[9, 1, 7, 5, 4, 7], 
        [1, 11, 4, 2, 7, 5], 
        [7, 4, 13, 5, 0, 7], 
        [5, 2, 5, 17, 1, 9], 
        [4, 7, 0, 1, 21, 15], 
        [7, 5, 7, 9, 15, 27]]
    C = np.array(C)
    return 5 + b@x + x @ (C @ x)

def gradV_a(x):
    b = np.array([1, 4, 5, 4, 2, 1])
    C = [[9, 1, 7, 5, 4, 7], 
        [1, 11, 4, 2, 7, 5], 
        [7, 4, 13, 5, 0, 7], 
        [5, 2, 5, 17, 1, 9], 
        [4, 7, 0, 1, 21, 15], 
        [7, 5, 7, 9, 15, 27]]
    C = np.array(C)
    return b + 2 * C @ x

In [5]:
b = np.array([1, 4, 5, 4, 2, 1])
C = [[9, 1, 7, 5, 4, 7], 
    [1, 11, 4, 2, 7, 5], 
    [7, 4, 13, 5, 0, 7], 
    [5, 2, 5, 17, 1, 9], 
    [4, 7, 0, 1, 21, 15], 
    [7, 5, 7, 9, 15, 27]]
C = np.array(C)
print(-np.linalg.solve(2*C, b), V_a(-np.linalg.solve(2*C, b)))

[ 0.33655646  0.05604031 -0.43004206 -0.19199735 -0.27131109  0.21006816] 3.654981973312939


In [7]:
#x0 = np.random.uniform(low=-5, high=5, size=(6,))
x0 = np.zeros((6,))
print(x0)

[0. 0. 0. 0. 0. 0.]


In [10]:
x, minimum =  gradient_descent(x0, V_a, gradV_a, step_size = 1e-4, log = True)

x = [ 0.27906133 -0.00838785 -0.32361257 -0.12871586 -0.13651301  0.06280731], V(x) = 3.94431
x = [ 0.28507953 -0.00508563 -0.3280806  -0.12923852 -0.13857687  0.06231429], V(x) = 3.94613
x = [ 0.28519593 -0.00502176 -0.32816702 -0.12924863 -0.13861679  0.06230475], V(x) = 3.94617
x = [ 0.28519818 -0.00502052 -0.32816869 -0.12924882 -0.13861756  0.06230457], V(x) = 3.94617


In [11]:
w = armijo(x0, V_a, gradV_a(x0), search_dir=-np.ones_like(x0), gamma=1.5, r=0.9, log=True)
print(w)
print(line_search_armijo(V_a, x0, np.ones_like(x0)*-1, gradV_a(x0), V_a(x0))[0])

p=0, q=32, w=0.038152042447694615
0.038152042447694615
0.034552845528455285


In [12]:
x, minimum =  conjugate_gradient(x0, V_a, gradV_a, step_size = 1e-4, log = False)
print(f'x = {x}, V(x) = {minimum:.5f}')

x = [ 0.28519823 -0.0050205  -0.32816872 -0.12924883 -0.13861758  0.06230457], V(x) = 3.94617


In [11]:
x, minimum = secant(x0,
           V_a,
           gradV_a,
           step_size = 1e-4, 
           )
x, minimum

(array([ 0.33655642,  0.0560403 , -0.43004201, -0.19199732, -0.27131106,
         0.21006813]),
 3.654981973312961)

In [ ]:
C = [[9, 1, 7, 5, 4, 7], 
    [1, 11, 4, 2, 7, 5], 
    [7, 4, 13, 5, 0, 7], 
    [5, 2, 5, 17, 1, 9], 
    [4, 7, 0, 1, 21, 15], 
    [7, 5, 7, 9, 5, 27]]
C = np.array(C)

print(np.linalg.inv(C))

[[ 0.29035961  0.09496866 -0.17677744 -0.04935856 -0.08911946  0.01892965]
 [ 0.08909692  0.16111106 -0.10081464 -0.02301464 -0.07493712  0.02250583]
 [-0.15606533 -0.0878367   0.20346969  0.01230757  0.06735968 -0.03754853]
 [-0.03067527 -0.00797569  0.00055336  0.08026624  0.01018591 -0.02312789]
 [-0.06245512 -0.05840609  0.06798228  0.02587966  0.08939641 -0.04890825]
 [-0.02952561 -0.0182099  -0.0010246  -0.01768016 -0.00043152  0.05446279]]


## Cost Function B

In [ ]:
def V_b(x):
    x1, x2 = x
    num = ((x1**2 + 1)*(2*x2**2 + 1))**0.5
    den = x1**2 + x2**2 + 0.5
    return -num / den

def gradV_b(x):
    x1, x2 = x

    num = (-x1**3 + x1*x2**2 - 1.5*x1)*(2*x2**2+1)**0.5
    den = (x1**2 + x2**2 + 0.5)**2 * (x1**2 + 1)**0.5
    dx1 = -num / den

    num = (-2*x2**3 + 2*x2*x1**2 - x2)*(x1**2+1)**0.5
    den = (x1**2 + x2**2 + 0.5)**2 * (2*x2**2 + 1)**0.5
    dx2 = -num / den

    return np.array([x1,x2])
    

In [ ]:
x0 = np.random.uniform(low=-5, high=5, size=(2,))

In [ ]:
x, minimum =  gradient_descent(x0, V_b, gradV_b, step_size = 1e-4, log = True)

x = [ 1.05575779 -1.72164   ], V(x) = -0.83596
x = [ 0.38837216 -0.63332429], V(x) = -1.36905
x = [ 0.14286699 -0.23297534], V(x) = -1.85069
x = [ 0.0525552  -0.08570255], V(x) = -1.97744
x = [ 0.01933301 -0.03152663], V(x) = -1.99690
x = [ 0.00711186 -0.01159742], V(x) = -1.99958
x = [ 0.00261618 -0.00426624], V(x) = -1.99994
x = [ 0.00096239 -0.00156938], V(x) = -1.99999
x = [ 0.00035403 -0.00057731], V(x) = -2.00000
x = [ 0.00013023 -0.00021237], V(x) = -2.00000
x = [ 4.79073681e-05 -7.81232612e-05], V(x) = -2.00000
x = [ 1.76232546e-05 -2.87385046e-05], V(x) = -2.00000
x = [ 6.48290886e-06 -1.05717764e-05], V(x) = -2.00000
x = [ 2.38480964e-06 -3.88894472e-06], V(x) = -2.00000
x = [ 8.77278569e-07 -1.43059128e-06], V(x) = -2.00000
x = [ 3.22716612e-07 -5.26258804e-07], V(x) = -2.00000
x = [ 1.18714871e-07 -1.93590114e-07], V(x) = -2.00000
x = [ 4.36705766e-08 -7.12142620e-08], V(x) = -2.00000
x = [ 1.60647040e-08 -2.61969529e-08], V(x) = -2.00000
x = [ 5.90957882e-09 -9.63683852e-0

In [ ]:
x, minimum =  conjugate_gradient(x0, V_b, gradV_b, step_size = 1e-4, log = False)
print(f'x = {x}, V(x) = {minimum:.5f}')

x = [ 5.22722522e-09 -8.52411430e-09], V(x) = -2.00000


In [ ]:
def augmented_lagrange(p, x0, tol=1e-6, tol_const=1e-6, sigma_max=1e12, hist=False):

    def phi(x, cost_function, lmb, sgm):
        cost = cost_function(x)

        n_e = p.num_eq_const()
        n_i = p.num_ineq_const()
        n_c = n_e + n_i

        lmb_e = lmb[0:n_e, :]
        lmb_i = lmb[n_e:n_c, :]
        sgm_e = sgm[0:n_e, :]
        sgm_i = sgm[n_e:n_c, :]

        if p.eq_const() is not None:
            c_e = p.eq_const(x)
            cost = cost - sum(lmb_e * c_e) + 0.5 * sum(sgm_e * c_e**2)

        if p.ineq_const() is not None:
            c_i = p.ineq_const(x)
            p_i = np.array([-lmb_i[i] * c_i[i] + 0.5 * sgm_i[i] * c_i[i]**2 \
                            if c_i[i] <= lmb_i[i] / sgm_i[i] \
                            else -0.5 * lmb_i[i]**2 / sgm_i[i] \
                            for i in range(0, n_i)])
            cost = cost + sum(p_i)

        return cost

    x_hist = []

    n_e = p.num_eq_const()
    n_i = p.num_ineq_const()
    n_c = n_e + n_i

    lmb = np.zeros((n_c, 1))
    sgm = np.ones((n_c, 1))

    x = x0
    c = 1e12 * np.ones((n_c, 1))

    while np.linalg.norm(c) > tol_const:
        # Create new problem to solve, but unconstrained
        up = Problem(partial(phi, p, lmb, sgm))
        x_hist.append(x)
        x = steepest_descent(up, x0, tol=tol)

        # Concatenate costs
        c_prv = c
        c_e = p.eq_const(x)
        c_i = p.ineq_const(x)
        if c_e is not None and c_i is not None:
            c = np.concatenate((c_e, c_i), axis=0)
        elif c_e is not None:
            c = c_e
        elif c_i is not None:
            c = c_i

        # Make sure sigma is not too big
        if any(sgm >= sigma_max):
            break

        # Update sigma
        if np.linalg.norm(c, np.inf) > 0.25 * np.linalg.norm(c_prv, np.inf):
            for i in range(0, n_c):
                if np.abs(c[i]) > 0.25 * np.linalg.norm(c_prv, np.inf):
                    sgm[i] *= 10
            continue

        lmb = lmb - (sgm * c)

    return x if not hist else np.array(x_hist)